In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')
import random as rnd
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
import matplotlib.patches as mpatches
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
%matplotlib inline
SEED=4242
rnd.seed(10)
SIZE=30000
ALPHA=0.03
# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import collections
from sklearn.ensemble import RandomForestClassifier
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score,\
 accuracy_score, classification_report, make_scorer, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_validate
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.under_sampling import TomekLinks
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from xgboost import plot_importance
# Time Series Features
# import tsfresh as tsf
from datetime import date, timedelta
from scipy import signal
from scipy.stats import variation
# Imbalance handling
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTETomek

In [56]:
df= pd.read_csv('/content/DFORE488.zip', compression='zip', low_memory=True)

In [57]:
print(df.shape)
df.drop_duplicates(subset= 'email_address', keep='last', inplace= True)
print(df.shape)
df.set_index('email_address', inplace=True)
bdex= list(filter(lambda x: '-' in x, df.columns.tolist()))
df['gender']= df.gender.map({'M':0, 'F':0})
bdex.extend(['gender', 'birth_year'])
bf= df[bdex]
rf= df[df.columns.difference(bdex)]
print('BDEX Columns: ',bf.shape,'RFM Columns: ', rf.shape)


(75545, 663)
(75545, 663)
BDEX Columns:  (75545, 608) RFM Columns:  (75545, 54)


In [58]:
df.bdex_data_flag.value_counts()

2    69429
1     4169
3     1395
0      552
Name: bdex_data_flag, dtype: int64

In [59]:
good=df.loc[df.bdex_data_flag== 1].index.to_list()
bad= df.loc[df.bdex_data_flag== 0].index.to_list()

In [60]:
y=pd.Series(1, index=good).append(pd.Series(0, index=bad))

In [61]:
y.value_counts()

1    4169
0     552
dtype: int64

In [62]:
select= y.index

## RFM

In [63]:
rf['Recency']= rf[['Vend_Recency', 'WalletLoad_Recency']].min(axis=1)
rf['DaysBetween']= rf[['AvgDaysBetweenVends', 'AvgDaysBetweenWalletLoad']].min(axis=1)
rf['DoB']= rf[['Vend_DaysonBook','WalletLoad_DaysonBook']].min(axis=1)
print(rf[['Recency', 'DaysBetween', 'DoB']].isnull().sum())
# Consumers in 150-180 cohort who visit atleast once in 10 days and their last activity is maximum 5 days old
# flg1= (rf.Recency <= 5) & (rf.DaysBetween<=10) & (rf.DoB > 150) & (rf.DoB <= 180)
# flg0= (rf.DoB > 150) & (rf.DoB <= 180)
# print('Target Group: ', sum(flg1))
# print('Size of Consumer Base: ', sum(flg0))
# print('Minority Class Group: ', 100.00*sum(flg1)/sum(flg0))


Recency        0
DaysBetween    0
DoB            0
dtype: int64


## BDEX

In [64]:
train = bf.loc[select].fillna(0)

In [65]:
# Our data is already scaled we should split our training and test sets
from sklearn.model_selection import train_test_split

# This is explicitly used for undersampling.
X_train, X_test, y_train, y_test = train_test_split(train.values, y.values, test_size=0.3, random_state=4242, shuffle=True)

print('Training Sample- Minority Class: ', sum(y_train))
print('Total Training Sample: ', len(y_train))
print('%: ', 100*sum(y_train)/len(y_train))
print()
print('Training Sample- Minority Class: ', sum(y_test))
print('Total Training Sample: ', len(y_test))
print('%: ', 100*sum(y_test)/len(y_test))

Training Sample- Minority Class:  2938
Total Training Sample:  3304
%:  88.9225181598063

Training Sample- Minority Class:  1231
Total Training Sample:  1417
%:  86.87367678193367


In [67]:
%%time
from pandas.core.apply import ResamplerWindowApply
datasets={}
resamplers= {'Random under-sampling': RandomUnderSampler(),
             'Random over-sampling': RandomOverSampler(),
             'Tomek links under-sampling': TomekLinks(sampling_strategy='majority'),
             'Cluster Centroids under-sampling': ClusterCentroids(),
             'SMOTE over-sampling': SMOTE(sampling_strategy='minority')}

for name, resampler in resamplers.items():
  X_retrain,y_retrain= resampler.fit_resample(X_train, y_train)
  datasets[name]=[X_retrain,y_retrain,X_test, y_test]
  print(name,X_retrain.shape,y_retrain.shape)

Random under-sampling (732, 608) (732,)
Random over-sampling (5876, 608) (5876,)
Tomek links under-sampling (3209, 608) (3209,)
Cluster Centroids under-sampling (732, 608) (732,)
SMOTE over-sampling (5876, 608) (5876,)
CPU times: user 38.9 s, sys: 5.99 s, total: 44.9 s
Wall time: 25.7 s


In [68]:
from pandas.core.common import random_state
classifiers = {
    "LogisiticRegression": LogisticRegression(class_weight='balanced', random_state=SEED),
    "KNearest": KNeighborsClassifier(),
    # "Support Vector Classifier": SVC(class_weight='balanced', probability=True, random_state=SEED),
    "DecisionTreeClassifier": DecisionTreeClassifier(class_weight='balanced', random_state=SEED),
    "RandomForest": RandomForestClassifier(class_weight= 'balanced', random_state=SEED),
    "XGBClassifier": xgb.XGBRFClassifier(tree_method="hist", random_state=SEED)
}
scorer={'f1_scorer': make_scorer(f1_score),
        'auc_scorer': make_scorer(roc_auc_score),
        'precision': make_scorer(precision_score),
        'recall': make_scorer(recall_score)}

In [69]:
%%time
models={}
recs=[]
for dk, v in datasets.items():
  for key, classifier in classifiers.items():
    print(dk, key)
    clear_output(wait= True)
    classifier.fit(v[0], v[1])
    training_score = cross_validate(classifier,v[0], v[1], cv=5, scoring= scorer)
    y_hat= classifier.predict(v[2])
    rec=[dk+': '+key, dk, key]
    rec.extend(list(np.array(list(training_score.values())).mean(axis=1)[2:]))
    # # roc_auc_score(v[3], classifier.predict_proba(v[2])[:, 1])
    rec.extend([f1_score(v[3], y_hat),roc_auc_score(v[3], classifier.predict_proba(v[2])[:, 1]), precision_score(v[3], y_hat), recall_score(y_hat, v[3])])
    recs.append(rec)
    # models[dk+': '+key]= [classifier, v[0], v[1], v[2], v[4], v[5]]

CPU times: user 1min 38s, sys: 6.98 s, total: 1min 45s
Wall time: 1min 31s


In [70]:
results= pd.DataFrame.from_records(recs)
results.columns=['DataClassifier', 'Sampling Strategy', 'Classifiers', 'train_f1', 'train_AUC', 'train_precision', 'train_recall','test_f1', 'test_AUC', 'test_precision', 'test_recall']

In [75]:
results.sort_values('train_AUC', ascending= False).head()

,DataClassifier,Sampling Strategy,Classifiers,train_f1,train_AUC,train_precision,train_recall,test_f1,test_AUC,test_precision,test_recall
23,SMOTE over-sampling: RandomForest,SMOTE over-sampling,RandomForest,0.909397,0.900641,0.921049,0.914556,0.902038,0.521276,0.871310,0.871310
8,Random over-sampling: RandomForest,Random over-sampling,RandomForest,0.891058,0.899074,0.967183,0.826400,0.865487,0.523894,0.876667,0.876667
7,Random over-sampling: DecisionTreeClassifier,Random over-sampling,DecisionTreeClassifier,0.875419,0.885973,0.964903,0.801222,0.838819,0.506831,0.872695,0.872695
19,Cluster Centroids under-sampling: XGBClassifier,Cluster Centroids under-sampling,XGBClassifier,0.853295,0.875546,0.994118,0.756498,0.012914,0.509541,1.000000,1.000000
22,SMOTE over-sampling: DecisionTreeClassifier,SMOTE over-sampling,DecisionTreeClassifier,0.871670,0.867279,0.916546,0.849875,0.868819,0.519020,0.874178,0.874178
